# Recording Audio from Computer


In [2]:
from sys import byteorder
from array import array
from struct import pack

import pyaudio
import wave


## audio capture code from : https://stackoverflow.com/questions/892199/detect-record-audio-in-python
THRESHOLD = 500
CHUNK_SIZE = 1024*3 #(3 seconds)
FORMAT = pyaudio.paInt16
RATE = 44100


def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.

    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > 30:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    #r = normalize(r)
    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()
    
##### Annie Code for iterating over data collection
#map sound name to it's number of data so far and it's shortcut
class Sound:
    def __init__(self, name, shortcut, count=0):
        self.name = name
        self.shortcut = shortcut
        self.count = 0

sounds = {'me':Sound("MicrowaveEnding","me"),
          'mo':Sound("MicrowaveDoorOpen","mo"),
          'mc':Sound("MicrowaveDoorClose","mc"),
          's':Sound("Sizzle","s"),
          'b':Sound("Boiling","b"),
          't':Sound("Toaster","m"),
          'f':Sound("FireAlarm","f"),
          'o':Sound("OvenTimer","o"),
          'g':Sound("GarbageDisposal","g"),
          'i':Sound("InstantPotBeep","i"),
          'fr':Sound("FridgeDoor","fr")}

    
def capture_audio():
    while 1:
        prompt = "What sound?\n"
        for sound in sounds.values():
            prompt += sound.shortcut + ": " + sound.name +"; "
        prompt += "\n x: Exit\n"
        soundNameID = input(prompt)
        if soundNameID == "x":
            return
        outfile = sounds[soundNameID].name+"_"+str(sounds[soundNameID].count)+"_captured.wav"
        sounds[soundNameID].count += 1
        print("recording\n")
        record_to_file(outfile)
        print("saved to: "+outfile)
    
if __name__ == '__main__':
    prompt = "What sound: "
    
    #for sound in Sounds:
    #    prompt += str(sound)
    #    print(sound)
    capture_audio()


What sound?
me: MicrowaveEnding; mo: MicrowaveDoorOpen; mc: MicrowaveDoorClose; s: Sizzle; b: Boiling; m: Toaster; f: FireAlarm; o: OvenTimer; g: GarbageDisposal; i: InstantPotBeep; fr: FridgeDoor; 
 x: Exit
x


# Sandbox

In [2]:
# https://stackoverflow.com/questions/892199/detect-record-audio-in-python
import pyaudio
import wave


# http://people.csail.mit.edu/hubert/pyaudio/
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


   # play with messaging 



In [14]:
import serial 
#import serial

arduinoData = serial.Serial('/dev/cu.usbmodem1411',9600)
def led_on():
    arduinoData.write(b'1')
    
def led_off():
    arduinoData.write(b'0')
    
t =0
while(t<4000000):
    t+=1
led_off()
print("done")
    

done
